In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import optuna
from sklearn.model_selection import cross_val_score
import pickle
import lightgbm as lgb


In [36]:
import mlflow
import dagshub
dagshub.init(repo_owner='satyajeetrai007', repo_name='Youtube-Comment-Sentiment-Analysis', mlflow=True)

Initialized MLflow to track repo "satyajeetrai007/Youtube-Comment-Sentiment-Analysis"

Repository satyajeetrai007/Youtube-Comment-Sentiment-Analysis initialized!

In [58]:
mlflow.set_experiment("lightGBM HPT on Custom Features")

2025/08/28 23:28:45 INFO mlflow.tracking.fluent: Experiment with name 'lightGBM HPT on Custom Features' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/2f1342ca992445ed9a0d602008d1373c', creation_time=1756403926717, experiment_id='11', last_update_time=1756403926717, lifecycle_stage='active', name='lightGBM HPT on Custom Features', tags={}>

In [38]:
# Load dataset
dataset = pd.read_csv('data_preprocessed.csv')

# Drop rows with NaN values in 'clean_comment'
cleaned_dataset = dataset.dropna().drop_duplicates()

In [39]:
# Separate features and target
X_cleaned = cleaned_dataset['clean_comment']
y_cleaned = cleaned_dataset['category']

# Split the cleaned data into train and test sets (80-20 split)
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)


In [40]:
# Load spacy language model for POS tagging
nlp = spacy.load('en_core_web_sm')

In [41]:
# All POS tags we care about
ALL_POS_TAGS = ['NOUN', 'VERB', 'ADJ', 'ADV', 'PRON', 'PROPN', 'INTJ', 'NUM', 'DET', 'ADP', 'CCONJ', 'PART', 'SCONJ', 'SYM', 'X', 'PUNCT', 'SPACE']

def extract_custom_features(text):
    doc = nlp(text)
    word_list = [token.text for token in doc]

    comment_length = len(text)
    word_count = len(word_list)
    avg_word_length = sum(len(word) for word in word_list) / word_count if word_count > 0 else 0
    unique_word_count = len(set(word_list))
    lexical_diversity = unique_word_count / word_count if word_count > 0 else 0

    pos_tags = [token.pos_ for token in doc]
    pos_proportion = {tag: pos_tags.count(tag) / word_count for tag in ALL_POS_TAGS}
    
    return {
        'comment_length': comment_length,
        'word_count': word_count,
        'avg_word_length': avg_word_length,
        'unique_word_count': unique_word_count,
        'lexical_diversity': lexical_diversity,
        **pos_proportion
    }


In [42]:
# Apply the custom feature extraction
train_custom_features = pd.DataFrame([extract_custom_features(text) for text in X_train_cleaned])
test_custom_features = pd.DataFrame([extract_custom_features(text) for text in X_test_cleaned])

In [43]:
train_custom_features.shape, test_custom_features.shape

((28994, 22), (7249, 22))

In [44]:
# Replace NaN values in POS tag proportions with 0
train_custom_features.fillna(0, inplace=True)
test_custom_features.fillna(0, inplace=True)

In [45]:
# Apply TfidfVectorizer with trigram setting and max_features=1000
tfidf = TfidfVectorizer(ngram_range=(1, 3), max_features=2000)
X_train_tfidf = tfidf.fit_transform(X_train_cleaned)
X_test_tfidf = tfidf.transform(X_test_cleaned)

In [46]:
# Convert TF-IDF to DataFrame
X_train_tfidf_df = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf.get_feature_names_out())
X_test_tfidf_df = pd.DataFrame(X_test_tfidf.toarray(), columns=tfidf.get_feature_names_out())

In [47]:
# Combine TF-IDF and custom features
X_train_combined = pd.concat([X_train_tfidf_df.reset_index(drop=True), train_custom_features.reset_index(drop=True)], axis=1)
X_test_combined = pd.concat([X_test_tfidf_df.reset_index(drop=True), test_custom_features.reset_index(drop=True)], axis=1)

In [48]:
X_train_combined.shape, X_test_combined.shape

((28994, 2022), (7249, 2022))

In [59]:
# Function to log results in MLflow
from sklearn.metrics import classification_report, accuracy_score
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, params, trial_number):
    with mlflow.start_run():
        # Log model type and trial number
        run_name = f"Trial_{trial_number}_{model_name}_class_weight"
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Log hyperparameters
        for key, value in params.items():
            mlflow.log_param(key, value)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        train_cv_accuracy  =  cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
        mlflow.log_metric("train_cv_accuracy ", train_cv_accuracy )
        accuracy = float(accuracy_score(y_test, y_pred))
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
     
        with open(f"{model_name}_model", "wb") as f:
            pickle.dump(model, f)
        mlflow.log_artifact(f"{model_name}_model", "model")
        

        return accuracy




In [60]:
# Function to optimize LightGBM hyperparameters
import lightgbm as lgb
def objective(trial):
    # Define hyperparameters to be tuned
    param = {
        "objective": "multiclass",
        "num_class": 3,  # Assuming 3 categories (-1, 0, 1)
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1e-1),
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "metric": "multi_logloss",
        "is_unbalance": True,
        "class_weight": "balanced",
    }

    # Define the LightGBM model with the trial parameters
    model = lgb.LGBMClassifier(**param)

    # Perform cross-validation
    accuracy = log_mlflow("LightGBM", model, X_train_combined, X_test_combined, y_train_cleaned, y_test_cleaned, param, trial.number)

    # Return the average score across folds
    return accuracy

In [61]:
# Create an Optuna study to optimize the hyperparameters
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2025-08-28 23:28:58,529] A new study created in memory with name: no-name-03533f37-5143-465d-ba8e-e0a0287cbb20


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.074095 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 107438
[LightGBM] [Info] Number of data points in the train set: 28994, number of used features: 1954
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu

[I 2025-08-28 23:31:39,600] Trial 0 finished with value: 0.8370809766864395 and parameters: {'learning_rate': 0.08740308671893958, 'n_estimators': 397, 'max_depth': 15}. Best is trial 0 with value: 0.8370809766864395.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.073706 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 107438
[LightGBM] [Info] Number of data points in the train set: 28994, number of used features: 1954
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.073918 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 77001
[LightGBM] [Info] Number of data points in the train set: 19

[I 2025-08-28 23:33:22,288] Trial 1 finished with value: 0.6889226100151745 and parameters: {'learning_rate': 0.00900008708096594, 'n_estimators': 103, 'max_depth': 12}. Best is trial 0 with value: 0.8370809766864395.


...

[I 2025-08-29 04:54:35,203] Trial 47 finished with value: 0.8368050765622844 and parameters: {'learning_rate': 0.07938445534209333, 'n_estimators': 483, 'max_depth': 13}. Best is trial 32 with value: 0.8413574286108428.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.151043 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 107438
[LightGBM] [Info] Number of data points in the train set: 28994, number of used features: 1954
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.070038 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 77001
[LightGBM] [Info] Number of data points in the train set: 19329, number of used features: 1934
[LightGBM] [Info] 

[I 2025-08-29 04:56:06,960] Trial 48 finished with value: 0.8075596634018485 and parameters: {'learning_rate': 0.0999852618280191, 'n_estimators': 80, 'max_depth': 18}. Best is trial 32 with value: 0.8413574286108428.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.094379 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 107438
[LightGBM] [Info] Number of data points in the train set: 28994, number of used features: 1954
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu

[I 2025-08-29 04:58:07,556] Trial 49 finished with value: 0.8232859704786867 and parameters: {'learning_rate': 0.08480752988325226, 'n_estimators': 374, 'max_depth': 7}. Best is trial 32 with value: 0.8413574286108428.


In [63]:
model = lgb.LGBMClassifier(

    objective='multiclass',
    num_class=3,
    metric="multi_logloss",
    is_unbalance= True,
    class_weight= "balanced",
    reg_alpha= 0.1,  # L1 regularization
    reg_lambda= 0.1,  # L2 regularization
    learning_rate= 0.06631684007924099,
    max_depth= 18,
    n_estimators=449
)

In [64]:
# Fit the model on the resampled training data
model.fit(X_train_combined, y_train_cleaned)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.185734 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 107438
[LightGBM] [Info] Number of data points in the train set: 28994, number of used features: 1954
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,18
,learning_rate,0.06631684007924099
,n_estimators,449
,subsample_for_bin,200000
,objective,'multiclass'
,class_weight,'balanced'
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [65]:
# Predict on the test set
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test_combined)
accuracy = accuracy_score(y_test_cleaned, y_pred)
accuracy

0.8391502276176024

In [ ]:
from sklearn.metrics import classification_report
# Generate classification report
report = classification_report(y_test_cleaned, y_pred)
print(report)

              precision    recall  f1-score   support

          -1       0.75      0.74      0.74      1657
           0       0.83      0.94      0.88      2393
           1       0.90      0.81      0.86      3199

    accuracy                           0.84      7249
   macro avg       0.83      0.83      0.83      7249
weighted avg       0.84      0.84      0.84      7249

